# 3 Extract facial data from videos.

We use a range of libraries to extract facial data from the videos. The main library is [DeepFace](https://github.com/serengil/deepface) but we also try FER - [Facial Expression Recognition](https://github.com/justinshenk/fer).

`pip install deepface`



## 3.1 DeepFace



In [ ]:
from deepface import DeepFace
import pandas as pd
import pprint
import cv2
import os
import time


In [ ]:
videos_in = "..\\LookitLaughter.test\\"
videos_out = "..\\data\\1_interim\\"

test_video = "2UWdXP.joke1.rep2.take1.Peekaboo.mp4"
test_video = "6c6MZQ.joke1.rep1.take1.ThatsNotAHat.mp4"

test_image = "peekaboo.png"
test_image2 = "mother-and-baby-in-park.jpg"

In [ ]:
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe',
  'yolov8',
  'yunet',
]

result = DeepFace.analyze(img_path=test_image2, enforce_detection = False, detector_backend = backends[1])

for r in result:
    pprint.pprint(r)


In [ ]:
def createfacesdf():
    #creates a dataframe with the facial data from the videos
    cols = ['frame', 'person', "bbox.x", "bbox.y","bbox.w","bbox.h","emotion","age","gender" ] #, "allemotions","allgenders"]
    return pd.DataFrame(columns=cols)

def addfacestodf(facesdf,frameidx, facedata):
    for idx, face in enumerate(facedata):
        newrow = {'frame': frameidx,
                  'person':idx,
                  'bbox.x':face['region']['x'],
                  'bbox.y':face['region']['y'],
                  'bbox.w':face['region']['w'],
                  'bbox.h':face['region']['h'],
                  'emotion':face['dominant_emotion'],
                  'age':face['age'],
                  'gender':face['dominant_gender']}
        #print(newrow)
        facesdf.loc[len(facesdf)] = newrow
    return facesdf

In [86]:
def getfacedataforvideo(video_path, backend = 'ssd'):
    print("Processing video: ", video_path)
    cap = cv2.VideoCapture(video_path)
    success, image = cap.read()
    facesdf = createfacesdf()
    frameidx = 0

    while success:
        try:  
            faces = DeepFace.analyze(img_path = image, 
                                    enforce_detection = False, 
                                    actions = ('age','gender','emotion'),
                                    detector_backend = backend)
        except:
            print(f"deepface.analyse failed for frame {frame}" )
            faces = []
        if len(faces) > 0:
            facesdf = addfacestodf(facesdf,frameidx,faces)
        success,image = cap.read()
        frameidx += 1
    
    cap.release()
    return facesdf



In [ ]:
def getprocessedvideos(videos_out):
    #check if we have already processed some videos
    if os.path.exists(videos_out + "\\processedvideos.xlsx"):
        print("found existing processedvideos.xlsx")
        processedvideos = pd.read_excel(videos_out + "\\processedvideos.xlsx")
    else:
        #create new dataframe for info about processed videos
        print("creating new processedvideos.xlsx")
        cols = ["VideoID","ChildID", "JokeType","JokeNum","JokeRep","JokeTake", "HowFunny","LaughYesNo", "Frames", "FPS", "Width", "Height", "Duration","Keypoints.when", "Keypoints.file","Audio.when","Audio.file","Emotions.when","Emotions.file","Speech.when","Speech.file","LastError"]
        processedvideos = pd.DataFrame(columns=cols)
    return processedvideos

In [ ]:
processedvideos = getprocessedvideos(videos_out)
processedvideos.head()

In [87]:
forceFaces = False
backend = "ssd"

for index, r in processedvideos.iterrows():
    if forceFaces or pd.isnull(r["Faces.file"]):
        facessdf = getfacedataforvideo(videos_in + r["VideoID"], backend = backend)
        facefile =  os.path.basename(r["VideoID"]) + f".faces.{backend}.csv"
        facessdf.to_csv(videos_out + facefile)
        r["Faces.file"] = videos_out + facefile
        r["Faces.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r
        

processedvideos.to_excel(videos_out + "\\processedvideos.xlsx")
processedvideos.head()

Processing video:  ..\LookitLaughter.test\4JDccE.joke1.rep1.take1.NomNomNom.mp4


Action: emotion: 100%|██████████| 3/3 [00:01<00:00,  2.54it/s]


deepface.analyse failed for frame {frame}
deepface.analyse failed for frame {frame}


Action: emotion: 100%|██████████| 3/3 [00:01<00:00,  2.42it/s]


deepface.analyse failed for frame {frame}
deepface.analyse failed for frame {frame}
deepface.analyse failed for frame {frame}
deepface.analyse failed for frame {frame}
deepface.analyse failed for frame {frame}


...

Processing video:  ..\LookitLaughter.test\6c6MZQ.joke5.rep1.take1.TearingPaper.mp4


Action: emotion: 100%|██████████| 3/3 [00:01<00:00,  2.05it/s]


Processing video:  ..\LookitLaughter.test\6c6MZQ.joke5.rep3.take1.TearingPaper.mp4


Action: emotion: 100%|██████████| 3/3 [00:01<00:00,  2.13it/s]


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,...,Duration,Keypoints.when,Keypoints.file,Audio.when,Audio.file,Faces.when,Faces.file,LastError,Speech.file,Speech.when
0,0,0,0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,...,15.175982,2023-09-19 11:39:38,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-22 14:03:28,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-22 18:33:10,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2023-09-20 16:58:38
1,1,1,1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,...,10.585630,2023-09-19 11:40:12,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-22 14:03:28,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-22 18:36:06,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2023-09-20 16:58:39
2,2,2,2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,1,1,Funny,...,7.174514,2023-09-19 11:40:36,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-22 14:03:29,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-22 18:37:57,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2023-09-20 16:58:40
3,3,3,3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,...,6.824348,2023-09-19 11:41:00,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-22 14:03:30,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-22 18:39:43,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2023-09-20 16:58:40
4,4,4,4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,...,9.350991,2023-09-19 11:41:34,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-22 14:03:30,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-22 18:42:27,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2023-09-20 16:58:48


In [82]:

processedvideos.to_excel(videos_out + "\\processedvideos.xlsx")
processedvideos.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,...,Duration,Keypoints.when,Keypoints.file,Audio.when,Audio.file,Faces.when,Faces.file,LastError,Speech.file,Speech.when
0,0,0,0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,...,15.175982,2023-09-19 11:39:38,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-22 14:03:28,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-22 18:33:10,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2023-09-20 16:58:38
1,1,1,1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,...,10.585630,2023-09-19 11:40:12,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-22 14:03:28,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-22 18:36:06,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2023-09-20 16:58:39
2,2,2,2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,1,1,Funny,...,7.174514,2023-09-19 11:40:36,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-22 14:03:29,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-22 18:37:57,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2023-09-20 16:58:40
3,3,3,3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,...,6.824348,2023-09-19 11:41:00,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-22 14:03:30,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-22 18:39:43,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2023-09-20 16:58:40
4,4,4,4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,...,9.350991,2023-09-19 11:41:34,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-22 14:03:30,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-22 18:42:27,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2023-09-20 16:58:48
